In [1]:
import json
import nltk

VISUAL = './scores/VISUAL-beam_2_cidertest.json'
LANGUAGE = './scores/LANGUAGE-beam_2_cidertest.json'
VISUAL_ATT = './scores/VISUAL_ATTENTION-beam_2_cidertest.json'
LANGUAGE_ATT = './scores/LANGUAGE_ATTENTION-beam_2_cidertest.json'
VISUAL_LANGUAGE = './scores/VISUAL_LANGUAGE-beam_2_cidertest.json'
VISUAL_LANGUAGE_ATT = './scores/VISUAL_LANG_ATTENTION_NOFUSION-beam_2_cidertest.json'

with open(VISUAL, 'r') as f:
    VISUAL = json.load(f)
with open(LANGUAGE, 'r') as l:
    LANGUAGE = json.load(l)
with open(VISUAL_ATT, 'r') as f:
    VISUAL_ATT = json.load(f)
with open(LANGUAGE_ATT, 'r') as l:
    LANGUAGE_ATT = json.load(l)
with open(VISUAL_LANGUAGE, 'r') as f:
    VISUAL_LANGUAGE = json.load(f)
with open(VISUAL_LANGUAGE_ATT, 'r') as l:
    VISUAL_LANGUAGE_ATT = json.load(l)
    
with open('/home/xilini/par-gen/01-par-gen/data/paragraphs_v1.json', 'r') as j:
    ground_truth = json.load(j)
with open('/home/xilini/par-gen/01-par-gen/data/splits.json', 'r') as s:
    splits = json.load(s)

gt = []
for elem in ground_truth:
    if splits[str(elem['image_id'])] == 'test':
        this_gt = {}
        this_gt['image_id'] = str(elem['image_id'])
        this_gt['hypotheses'] = elem['paragraph']
        gt.append(this_gt)
    

hypotheses = [
    VISUAL,
    VISUAL_ATT,
    LANGUAGE,
    LANGUAGE_ATT,
    VISUAL_LANGUAGE,
    VISUAL_LANGUAGE_ATT,
    gt
]


In [2]:
# Construction 1
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
# Create a blank Tokenizer with just the English vocab
tokenizer = Tokenizer(nlp.vocab)

In [15]:
def extract_hyp(data_dict):
    hyps_str = ''
    for item in data_dict:
        hyps_str += item['hypotheses']
        hyps_str += ' '
    #out = [w.text for w in tokenizer(hyps_str)]
    #out = nltk.tokenize.word_tokenize(hyps_str)
    return out

In [16]:
# vocabulary size (number of types)

for each_set in hypotheses:
    this_set_hyps_words = extract_hyp(each_set)
    
    print(len(this_set_hyps_words))
    print(len(set(this_set_hyps_words)))
    print()


93430
403

95993
390

91659
389

95047
376

95411
358



KeyboardInterrupt: 

In [ ]:
# number of nouns and unique nouns (nouns describe objects, so it is a rough indicator of object mention)

is_noun = lambda pos: pos[:2] == 'NN'
for each_set in hypotheses:
    this_set_hyps_words = extract_hyp(each_set)
    this_set_nouns = [word for (word, pos) in nltk.pos_tag(this_set_hyps_words) if is_noun(pos)]
    print(len(this_set_nouns))
    print(len(set(this_set_nouns)))
    print()

In [5]:
p = tokenizer('The big thing.')

In [7]:
len(p)

3

In [ ]:
# self BLEU

In [ ]:
import sys
sys.path.insert(0, '/home/xilini/par-gen/01-par-gen/')
from evalfunc.bleu.bleu import Bleu
import nltk

In [ ]:
def self_bleu(each_set):
    
    refs_all = {}
    hyps_all = {}
    
    for item in each_set:
        
        this_image_id = str(item['image_id'])
        this_hyps = item['hypotheses']
        sents = nltk.sent_tokenize(this_hyps)
        
        for ref_n, each_s in enumerate(sents):
            these_hyps = [each_s]
            these_ref = []
            for hyp_n, other_s in enumerate(sents):
                if hyp_n != ref_n:
                    these_ref.append(other_s)
            if these_ref != []:
                refs_all[this_image_id + '_' + str(ref_n)] = these_ref
                hyps_all[this_image_id + '_' + str(ref_n)] = these_hyps
            
    return refs_all, hyps_all

In [ ]:
scorers = [
    (Bleu(4), ["Bleu_1", "Bleu_2", "Bleu_3", "Bleu_4"]),
]

for each_set in hypotheses:
    
    refs, hyps = self_bleu(each_set)

    score = []
    method = []
    for scorer, method_i in scorers:

        score_i, _ = scorer.compute_score(refs, hyps)
        score.extend(score_i) if isinstance(score_i, list) else score.append(score_i)
        method.extend(method_i) if isinstance(method_i, list) else method.append(method_i)

    score_dict = dict(zip(method, score))
    
    print(score_dict)
    print()

In [ ]:
def repetetive_nouns(data_dict):
    all_ratios = 0
    for item in data_dict:
        hyp = item['hypotheses'].split('.')
        this_hyp_nouns = {}
        num_of_rep_nouns = 0
        for elem in hyp:
            out = nltk.tokenize.word_tokenize(elem)
            nouns = [word for (word, pos) in nltk.pos_tag(out) if is_noun(pos)]
            for n in nouns:
                if n not in this_hyp_nouns:
                    this_hyp_nouns[n] = 1
                else:
                    this_hyp_nouns[n] += 1
        for each_n, freq in this_hyp_nouns.items():
            if freq != 1:
                # if some noun repeats > 1 times
                num_of_rep_nouns += 1
        ratio = num_of_rep_nouns / len(this_hyp_nouns)
        all_ratios += ratio
    return all_ratios / len(data_dict)

In [ ]:
# average ratio of repetetive nouns / all unique nouns per paragraph
# smaller means more unique nouns in a generated paragraph on average
# nouns repeated more than once / all nouns

# 'in a single paragraph on average, 49% of the nouns are occuring more than 1 time in its sentences'
# models with multimodal input demonstrate more diversity in terms of nouns they contain
# this number should be high enough for connectivity, but not too much
# IDEA: introduce n-gram penalty on the level of words and/or the level of sentences?
# this would make av_rep_ratio_noun 'better', and improve generation evaluation scores

for each_set in hypotheses:
    av_rep_ratio_noun = repetetive_nouns(each_set)
    print(av_rep_ratio_noun)

|   |                                 | # of words | # of types | # of nouns | # of unique nouns | % of REP NOUNS |
|---|---------------------------------|------------|------------|------------|-------------------|-----------|
|   | BASELINE MULTIMODAL- ATTENTION- | 115,753    | 413        | 28,337     | 308               | 43        |
|   | BASELINE MULTIMODAL+ ATTENTION- | 116,381    | 437        | 28,755     | 321               | 42        |
|   | BASELINE MULTIMODAL- ATTENTION+ | 118,298    | 416        | 29,094     | 317               | 41        |
|   | BASELINE MULTIMODAL+ ATTENTION+ | 113,042    | 441        | 27,914     | 327               | 39        |

In [ ]:
def word_overlap(data_dict):
    
    item_count = 0
    item_count_not = 0
    
    # get all unique words from ground-truth paragraphs
    gt_str = ''
    for item in gt:
        gt_str += item['hypotheses']
        gt_str += ' '
    out = nltk.tokenize.word_tokenize(gt_str)
    out = set(out)
    
    # get all unique words in our generations
    item_str = ''
    for item in data_dict:
        item_str += item['hypotheses']
        item_str += ' '
    item_out = nltk.tokenize.word_tokenize(item_str)
    item_out = set(item_out)
    
    for i in item_out:
        if i in out:
            item_count += 1
        else:
            item_count_not += 1
            
    item_count = (item_count * 100) / len(out)
    
    return item_count, 100 - item_count

In [ ]:
# overlap in terms of generated words
# which model overlaps more with words in ground truth

hypotheses = [beam_nomulti, beam_multi, beam_nomulti_att, beam_multi_att]
for each_set in hypotheses:
    overlapped, not_overlapped = word_overlap(each_set)
    print(overlapped, not_overlapped)